In [1]:
!pip uninstall -y opencv-python opencv-python-headless numpy deepface deep-sort-realtime tensorflow torch torchvision torchaudio scikit-learn pinecone


The history saving thread hit an unexpected error (OperationalError('unable to open database file')).History will not be written to the database.
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Successfully uninstalled tensorflow-2.17.0
Found existing installation: torch 2.4.1.post100
Uninstalling torch-2.4.1.post100:
  Successfully uninstalled torch-2.4.1.post100
Found existing installation: torchvision 0.19.1
Uninstalling torchvision-0.19.1:
  Successfully uninstalled torchvision-0.19.1
Found existing installation: scikit-learn 1.5.2
Uninstalling scikit-learn-1.5.2:
  Successfully uninstalled scikit-learn-1.5.2


In [5]:
%%writefile drwa.py
import cv2
import numpy as np
from deepface import DeepFace
from deep_sort_realtime.deepsort_tracker import DeepSort

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30)

# Open the video file
cap = cv2.VideoCapture("classrooma.mp4")

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)  # Frames per second
start_frame = int(4 * fps)  # Start at 3 seconds
end_frame = int(8 * fps)  # End at 10 seconds

embeddings_dict = {}  # Dictionary to store embeddings {track_id: embedding_list}

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Skip frames before the start time
    if frame_count < start_frame:
        continue

    # Stop processing after the end time
    if frame_count > end_frame:
        break

    # Process every 5th frame for efficiency
    if frame_count % 5 == 0:
        try:
            # Resize frame for better performance
            frame_resized = cv2.resize(frame, (640, 360))

            # Detect faces
            faces = DeepFace.extract_faces(frame_resized, detector_backend="retinaface", enforce_detection=False)

            detections = []
            for face in faces:
                facial_area = face["facial_area"]
                x, y, w, h = facial_area["x"], facial_area["y"], facial_area["w"], facial_area["h"]
                bbox = [x, y, x + w, y + h]  # Convert to (x1, y1, x2, y2)

                detections.append((bbox, 1.0))  # (bounding_box, confidence_score)

            # Update tracker with detections
            tracked_faces = tracker.update_tracks(detections, frame=frame_resized)

            for track in tracked_faces:
                if not track.is_confirmed():
                    continue  # Ignore unconfirmed tracks

                track_id = track.track_id  # Unique ID assigned by DeepSORT
                bbox = track.to_ltrb()  # Get bounding box (x1, y1, x2, y2)

                # Crop the detected face
                x1, y1, x2, y2 = map(int, bbox)
                face_crop = frame_resized[y1:y2, x1:x2]

                # Ensure the crop is not empty
                if face_crop.size == 0:
                    continue

                # Get embedding for the tracked face
                embedding = DeepFace.represent(face_crop, model_name="Facenet", enforce_detection=False)
                if embedding:
                    embedding = embedding[0]["embedding"]
                    if track_id not in embeddings_dict:
                        embeddings_dict[track_id] = [np.array(embedding, dtype=np.float32)]
                    else:
                        embeddings_dict[track_id].append(np.array(embedding, dtype=np.float32))

        except Exception as e:
            print(f"Error processing frame {frame_count}: {e}")

# Average embeddings for each tracked person
final_embeddings = {track_id: np.mean(embeddings, axis=0) for track_id, embeddings in embeddings_dict.items()}

# Convert dictionary to a NumPy array and save
np.save("tracked_face_embeddings.npy", final_embeddings)
print(f"Saved {len(final_embeddings)} unique face embeddings to tracked_face_embeddings.npy")

# Cleanup
cap.release()
cv2.destroyAllWindows()

Overwriting drwa.py


In [6]:
import numpy as np

# Load the saved embeddings
embeddings = np.load("tracked_face_embeddings.npy", allow_pickle=True).item()

# Print the embeddings
print(embeddings)

{'1': array([-0.17045061,  0.3332695 ,  0.59908915, -0.7415862 , -0.10415559,
        0.12151986,  0.88161117, -0.65495825, -0.16447303,  0.80027705,
        0.2910854 ,  0.15943286,  0.13427003, -0.46339226,  0.17969814,
        0.5544893 , -0.18968946, -1.1058036 , -0.44528726, -0.51665455,
        0.43294275, -0.7521591 , -0.00263618,  0.29751927, -0.6230553 ,
       -0.1891846 ,  0.24976043,  0.20649855,  0.11538564,  0.30589646,
       -0.25506714, -0.20951882,  0.02272737, -0.5420021 ,  0.6142189 ,
        0.04159555, -0.97948915, -0.03390833,  0.22418304,  0.2616121 ,
       -0.26914346, -0.35589966, -0.36173147,  0.5020479 , -0.82086873,
       -0.48208815, -0.15118298,  0.13040118, -0.78610253, -0.48140424,
        0.00145244,  0.40780765,  0.11898398, -0.13268699,  0.11159035,
        0.31218326,  0.01776191,  0.24724084, -0.36590043, -0.13728108,
       -0.681348  , -0.10427935, -0.65236986, -0.2564089 ,  0.27837607,
       -0.24725068, -0.23050654,  0.33736554,  0.26800373,

In [1]:
%%writefile newdrwa.py
import cv2
import numpy as np
from pinecone import Pinecone
from deepface import DeepFace
from deep_sort_realtime.deepsort_tracker import DeepSort
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Initialize Pinecone
pc = Pinecone(api_key="pcsk_6m9iGK_SJCAHA6WqusNpxon1s3M9W6VAjfZk86H7mQrrGwQPA67G2nVkhkKLCWcDEas6Kv")
index = pc.Index("deepface")  # Ensure this index uses Euclidean distance

# Initialize DeepSORT tracker
tracker = DeepSort(max_age=30)

# Open the video file
cap = cv2.VideoCapture("classrooma.mp4")

# Get video properties
fps = cap.get(cv2.CAP_PROP_FPS)
start_frame = int(4 * fps)
end_frame = int(8 * fps)

frame_count = 0

# Dictionary to store embeddings for each tracked face {track_id: [embedding1, embedding2, ...]}
embeddings_dict = {}

def query_pinecone(embedding, index, threshold=0.8):
    """Query Pinecone for similar faces using Euclidean distance."""
    # Normalize the embedding
    embedding_normalized = embedding / np.linalg.norm(embedding)
    embedding_list = embedding_normalized.tolist()

    response = index.query(
        namespace="default",
        vector=embedding_list,
        top_k=1,  # Get the most similar face
        include_values=True,
        include_metadata=True
    )
    print(f"Pinecone Response: {response}")  # Debugging: Print the full response

    if response.get("matches"):
        best_match = response["matches"][0]
        print(f"Best Match Score: {best_match['score']}")  # Debugging: Print the score
        if best_match["score"] <= threshold:  # Euclidean distance: lower is better
            return best_match.get("metadata", {}).get("name", "Unknown"), best_match["score"]
    return None, 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Skip frames before start time
    if frame_count < start_frame:
        continue
    # Stop processing after the end time
    if frame_count > end_frame:
        break

    # Process every 5th frame for efficiency
    if frame_count % 5 == 0:
        try:
            frame_resized = cv2.resize(frame, (640, 360))

            # Detect faces
            faces = DeepFace.extract_faces(frame_resized, detector_backend="retinaface", enforce_detection=False)

            detections = []
            for face in faces:
                facial_area = face["facial_area"]
                x, y, w, h = facial_area["x"], facial_area["y"], facial_area["h"], facial_area["w"]
                bbox = [x, y, x + w, y + h]  # Convert to (x1, y1, x2, y2)
                detections.append((bbox, 1.0))  # (bounding_box, confidence_score)

            # Update tracker
            tracked_faces = tracker.update_tracks(detections, frame=frame_resized)

            for track in tracked_faces:
                if not track.is_confirmed():
                    continue  # Ignore unconfirmed tracks

                track_id = str(track.track_id)  # Convert track ID to string
                bbox = track.to_ltrb()  # Bounding box (x1, y1, x2, y2)
                x1, y1, x2, y2 = map(int, bbox)
                face_crop = frame_resized[y1:y2, x1:x2]

                if face_crop.size == 0 or face_crop.shape[0] < 10 or face_crop.shape[1] < 10:
                    continue  # Skip if no face detected or face is too small

                # Generate face embedding
                embedding = DeepFace.represent(face_crop, model_name="Facenet", enforce_detection=False)
                if embedding:
                    embedding = embedding[0]["embedding"]

                    # Store embedding in the dictionary
                    if track_id not in embeddings_dict:
                        embeddings_dict[track_id] = [np.array(embedding, dtype=np.float32)]
                    else:
                        embeddings_dict[track_id].append(np.array(embedding, dtype=np.float32))

        except Exception as e:
            logging.error(f"Error processing frame {frame_count}: {str(e)}")

# Average embeddings for each tracked face
final_embeddings = {track_id: np.mean(embeddings, axis=0) for track_id, embeddings in embeddings_dict.items()}

# Query Pinecone with averaged embeddings
for track_id, avg_embedding in final_embeddings.items():
    print(f"Averaged Embedding for Track ID {track_id}: {avg_embedding}")  # Debugging: Print the averaged embedding
    person_name, distance_score = query_pinecone(avg_embedding, index, threshold=0.8)
    if person_name:
        logging.info(f"Track ID {track_id} is {distance_score:.4f} Euclidean distance from {person_name}")
    else:
        logging.info(f"No similar face found in the database for Track ID {track_id}.")

# Cleanup
cap.release()
cv2.destroyAllWindows()

Overwriting newdrwa.py


In [2]:
!pip cache purge

Files removed: 543


In [3]:
!source venv/bin/activate


ModuleNotFoundError: No module named 'deepface'